<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Toxicity Classification Evals</h1>

## Install Dependencies and Import Libraries

In [ ]:
!pip install -qq "arize-phoenix[experimental]" ipython matplotlib openai pycm scikit-learn

In [ ]:
import os
from getpass import getpass

import matplotlib.pyplot as plt
import openai
import pandas as pd
from phoenix.experimental.evals import (
    TOXICITY_PROMPT_RAILS_MAP,
    TOXICITY_PROMPT_TEMPLATE_STR,
    OpenAiModel,
    llm_eval_binary,
)
from pycm import ConfusionMatrix
from sklearn.metrics import classification_report

## Download Benchmark Dataset

We'll evaluate the evaluation system consisting of an LLM model and settings in addition to an evaluation prompt template against benchmark datasets of queries and retrieved documents with ground-truth toxicity labels. Currently supported datasets include:

- "wiki_toxic"


In [ ]:
df = pd.read_parquet(
    "https://huggingface.co/api/datasets/OxAISH-AL-LLM/wiki_toxic/parquet/default/test/0.parquet"
)
df.head()

## Display Toxicity Classification Template

View the default template used to classify toxicity. You can tweak this template and evaluate its performance relative to the default.

In [ ]:
print(TOXICITY_PROMPT_TEMPLATE_STR)

The template variables are:

- **text:** the text provided by a user

# Configure the LLM

Configure your OpenAI API key.

In [ ]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

Instantiate the LLM and set parameters.

In [ ]:
model = OpenAiModel(
    model_name="gpt-4",
    temperature=0.0,
)

## Run Toxicity Classifications

Run toxicity classifications against a subset of the data.

In [ ]:
df = df.sample(n=100).reset_index(drop=True)
df = df.rename(
    columns={"comment_text": "text"},
)

In [ ]:
toxic_classifications = llm_eval_binary(
    dataframe=df.head(10),
    template=TOXICITY_PROMPT_TEMPLATE_STR,
    model=model,
    rails=TOXICITY_PROMPT_RAILS_MAP.values(),
)

## Evaluate Classifications

Evaluate the predictions against human-labeled ground-truth relevance labels.

In [ ]:
classes = TOXICITY_PROMPT_RAILS_MAP.values()
true_labels = df["toxic"].replace((True, False), classes).tolist()
predicted_labels = toxic_classifications

print(classification_report(true_labels, predicted_labels, labels=classes))
confusion_matrix = ConfusionMatrix(
    actual_vector=true_labels, predict_vector=predicted_labels, classes=classes
)
confusion_matrix.plot(
    cmap=plt.colormaps["Blues"],
    number_label=True,
    normalized=True,
);